# CNN

## Imports

In [1]:
import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../modules'))
import notebook_loading

from CNN_Train import CNN_Train
from CNN_Data import *
from CN_Network import CN_Network

importing Jupyter notebook from CNN_Train.ipynb
importing Jupyter notebook from CNN_Data.ipynb
importing Jupyter notebook from CN_Network.ipynb


## Load & Setup Data

In [2]:
# extract the redundant baselines and their gains and data from miriad and calfits files
red_bls, gains, uvd = load_relevant_data('../zen_data/zen.2458098.58037.xx.HH.uv','../zen_data/zen.2458098.58037.xx.HH.uv.abs.calfits')

# seperate trining and testing redundant baselines 
# if we have not already done this, load them from disk
training_redundant_baselines_dict, testing_redundant_baselines_dict = get_or_gen_test_train_red_bls_dicts(red_bls, gains.keys())

# seperate the visiblites
training_baselines_data = get_seps_data(training_redundant_baselines_dict, uvd)
testing_baselines_data = get_seps_data(testing_redundant_baselines_dict, uvd)

## Trainer

In [3]:
trainer = CNN_Train()

In [4]:
trainer.add_data((training_baselines_data, training_redundant_baselines_dict),
                 (testing_baselines_data, testing_redundant_baselines_dict),
                 gains)

## Network

In [5]:
wide_filter_widths = [64,32,8]
width_reduction_factors = [2,3,4]
learning_rate = 0.00005

example_network_A = CN_Network(name = 'example_network_A',
                             wide_filter_widths = wide_filter_widths,
                             width_reduction_factors = width_reduction_factors,
                             learning_rate = learning_rate)
example_network_A.params

{'cost_name': 'MSE',
 'dtype': tf.float32,
 'g_shift': 0.01,
 'learning_rate': 5e-05,
 'name': 'example_network_A',
 'num_freq_channels': 1024,
 'threshold': 0.00625,
 'wide_filter_widths': [64, 32, 8],
 'width_reduction_factors': [2, 3, 4]}

In [6]:
trainer.network = example_network_A
trainer.num_epochs = 2 # quick example
trainer.train()

Graph Ready
Training Finished


In [7]:
# train, test results
print(trainer.PWTs) 
print(trainer.MISGs)
print(trainer.MSEs)

[(1.5833333, 1.8333333), (0.8333334, 0.8166667)]
[(94.27098, 94.0534), (94.936356, 95.41085)]
[(0.11045755, 0.109611236), (0.08841964, 0.089243196)]


### Can use same trainer for another network

In [8]:
wide_filter_widths = [12,8,4]
width_reduction_factors = [2,2,2]
learning_rate = 0.00005

example_network_B = CN_Network(name = 'example_network_B',
                             wide_filter_widths = wide_filter_widths,
                             width_reduction_factors = width_reduction_factors,
                             learning_rate = learning_rate)
example_network_B.params

{'cost_name': 'MSE',
 'dtype': tf.float32,
 'g_shift': 0.01,
 'learning_rate': 5e-05,
 'name': 'example_network_B',
 'num_freq_channels': 1024,
 'threshold': 0.00625,
 'wide_filter_widths': [12, 8, 4],
 'width_reduction_factors': [2, 2, 2]}

In [9]:
trainer.network = example_network_B
trainer.num_epochs = 2 # quick example
trainer.train()

Graph Ready
Training Finished


In [10]:
# train, test results
print(trainer.PWTs) 
print(trainer.MISGs)
print(trainer.MSEs)

[(1.6166667, 1.5833333), (1.4166667, 0.6333333)]
[(94.20609, 94.65782), (94.77652, 95.850105)]
[(0.085117735, 0.08686262), (0.0843562, 0.087568715)]
